# Branch Assessment
Project created and best viewed in [Jupyter Notebooks](https://www.anaconda.com/products/individual)

[Entity Relationship Diagram](https://dbdiagram.io/d/61fd6ea885022f4ee53ec996)


### My assumptions for this project
- There was data governance need to break up the data into separate tables. With no imposed restrictions, I would prefer to store the JSON response as a whole and implementing a schema-on-read solution instead of a legacy RDBMS solution like this which would change the workflow from ETL to ELT
- The UUID could be trusted to be used as a true universally unique identifier
- Dimensions were to be SCD type 1 (overwritten when updated, no history)
- Storing the password and identity/ssn info in plain text was bait, so I only stored the hashed versions of those attributes

### Other things worth mentioning
- I might have created an enum type to whitelist possible identity types if I knew all possible options
- [Learn more about me](shorturl.at/ruRV8)
- Deliverable 3 is in the form of the production-etl-notes.txt file

In [5]:
import pandas
import numpy
import json
import os
import hashlib
from urllib.request import urlopen

source_url = 'https://randomuser.me/api/?results=500'
source_response = urlopen(source_url)
source_data = json.loads(source_response.read())

df = pandas.json_normalize(source_data['results'])

#
# Create new column in dataframe for hashed identity/ssn
#
df['hash'] = numpy.where(df['id.value'].isnull(), '', df['id.value'].astype(str).apply(lambda x: hashlib.sha256(x.encode()).hexdigest()))


#
# Column naming
#
df.columns
df.rename(columns={
    'login.uuid': 'uuid',
    'login.username': 'username',
    'login.salt': 'salt',
    'login.sha256': 'sha256',
    'location.street.number': 'street_number',
    'location.street.name': 'street_name',
    'location.city': 'city',
    'location.state': 'state',
    'location.country': 'country',
    'location.postcode': 'postal_code',
    'location.coordinates.latitude': 'lat',
    'location.coordinates.longitude': 'long',
    'location.timezone.offset': 'utc_offset',
    'location.timezone.description': 'utc_description',
    'name.title': 'name_salutation',
    'name.first': 'name_first',
    'name.last': 'name_last',
    'dob.date': 'dob',
    'picture.thumbnail': 'picture_thumbnail',
    'picture.medium': 'picture_medium',
    'picture.large': 'picture_large',
    'registered.date': 'date_registered',
    'id.name': 'type'
}, inplace=True)


#
# Create dataframes for each table 
#

auth_table = df[[
    'uuid',
    'username',
    'salt',
    'sha256'
]]

locations_table = df[[
    'uuid',
    'street_number',
    'street_name',
    'city',
    'state',
    'country',
    'postal_code',
    'lat',
    'long',
    'utc_offset',
    'utc_description'
]]

user_details_table = df[[
    'uuid',
    'gender',
    'name_salutation',
    'name_first',
    'name_last',
    'email',
    'dob',
    'phone',
    'cell',
    'picture_thumbnail',
    'picture_medium',
    'picture_large',
    'date_registered'
]]

identities_table = df[[
    'uuid',
    'type',
    'hash'
]]

auth_table.to_csv('csvs/auth_table.csv')
locations_table.to_csv('csvs/locations_table.csv')
user_details_table.to_csv('csvs/user_details_table.csv')
identities_table.to_csv('csvs/identities_table.csv')